In [1]:
import random
import sys
import os

import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import LambdaCallback

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, Dropout
from tensorflow.keras.layers import LSTM

%matplotlib inline

# a custom data prep class that we'll be using 
!wget https://raw.githubusercontent.com/LambdaSchool/DS-Unit-4-Sprint-3-Deep-Learning/main/module1-rnn-and-lstm/data_cleaning_toolkit_class.py
from data_cleaning_toolkit_class import data_cleaning_toolkit

2021-10-19 16:41:19.362196: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-19 16:41:19.362218: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


--2021-10-19 16:41:23--  https://raw.githubusercontent.com/LambdaSchool/DS-Unit-4-Sprint-3-Deep-Learning/main/module1-rnn-and-lstm/data_cleaning_toolkit_class.py
SSL_INIT
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6666 (6.5K) [text/plain]
Saving to: ‘data_cleaning_toolkit_class.py.3’

data_cleaning_toolk 100%[===================>]   6.51K  --.-KB/s    in 0s      

2021-10-19 16:41:24 (46.1 MB/s) - ‘data_cleaning_toolkit_class.py.3’ saved [6666/6666]



### Use request to pull data from a URL

[**Read through the request documentation**](https://requests.readthedocs.io/en/master/user/quickstart/#make-a-request) to learn how to download the Shakespeare Sonnets from the Gutenberg website. 

**Protip:** Do not overthink it.

In [2]:
# download all of Shakespear's Sonnets from the Project Gutenberg website

# here's the link for the sonnets
url_shakespeare_sonnets = "https://www.gutenberg.org/cache/epub/1041/pg1041.txt"

# use request and the url to download all of the sonnets - save the result to `r`



# YOUR CODE HERE
r = requests.get(url_shakespeare_sonnets)

In [3]:
dir(r)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [4]:
# move the downloaded text out of the request object - save the result to `raw_text_data`
# hint: take a look at the attributes of `r`
# YOUR CODE HERE
raw_text_data = r.text

In [5]:
raw_text_data

'\ufeffThe Project Gutenberg EBook of Shakespeare\'s Sonnets, by William Shakespeare\r\n\r\nThis eBook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  You may copy it, give it away or\r\nre-use it under the terms of the Project Gutenberg License included\r\nwith this eBook or online at www.gutenberg.org\r\n\r\n\r\nTitle: Shakespeare\'s Sonnets\r\n\r\nAuthor: William Shakespeare\r\n\r\nPosting Date: April 7, 2014 [EBook #1041]\r\nRelease Date: September, 1997\r\nLast Updated: March 10, 2010\r\n\r\nLanguage: English\r\n\r\n\r\n*** START OF THIS PROJECT GUTENBERG EBOOK SHAKESPEARE\'S SONNETS ***\r\n\r\n\r\n\r\n\r\nProduced by Joseph S. Miller and Embry-Riddle Aeronautical\r\nUniversity Library. HTML version by Al Haines.\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nTHE SONNETS\r\n\r\nby William Shakespeare\r\n\r\n\r\n\r\n\r\n  I\r\n\r\n  From fairest creatures we desire increase,\r\n  That thereby beauty\'s rose might never die,\r\n  But as the r

In [6]:
# check the data type of `raw_text_data`
type(raw_text_data)

str

### Data Cleaning

In [7]:
# as usual, we are tasked with cleaning up messy data
# Question: Do you see any characters that we could use to split up the text?
raw_text_data[:3000]

"\ufeffThe Project Gutenberg EBook of Shakespeare's Sonnets, by William Shakespeare\r\n\r\nThis eBook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  You may copy it, give it away or\r\nre-use it under the terms of the Project Gutenberg License included\r\nwith this eBook or online at www.gutenberg.org\r\n\r\n\r\nTitle: Shakespeare's Sonnets\r\n\r\nAuthor: William Shakespeare\r\n\r\nPosting Date: April 7, 2014 [EBook #1041]\r\nRelease Date: September, 1997\r\nLast Updated: March 10, 2010\r\n\r\nLanguage: English\r\n\r\n\r\n*** START OF THIS PROJECT GUTENBERG EBOOK SHAKESPEARE'S SONNETS ***\r\n\r\n\r\n\r\n\r\nProduced by Joseph S. Miller and Embry-Riddle Aeronautical\r\nUniversity Library. HTML version by Al Haines.\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nTHE SONNETS\r\n\r\nby William Shakespeare\r\n\r\n\r\n\r\n\r\n  I\r\n\r\n  From fairest creatures we desire increase,\r\n  That thereby beauty's rose might never die,\r\n  But as the riper

In [8]:
# split the text into lines and save the result to `split_data`
# YOUR CODE HERE
split_data = raw_text_data.split('\r\n')

In [9]:
# we need to drop all the boilerplate text (i.e., titles and descriptions) as well as white spaces
# so that we are left with only the sonnets themselves 
split_data[:20] 

["\ufeffThe Project Gutenberg EBook of Shakespeare's Sonnets, by William Shakespeare",
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included',
 'with this eBook or online at www.gutenberg.org',
 '',
 '',
 "Title: Shakespeare's Sonnets",
 '',
 'Author: William Shakespeare',
 '',
 'Posting Date: April 7, 2014 [EBook #1041]',
 'Release Date: September, 1997',
 'Last Updated: March 10, 2010',
 '',
 'Language: English',
 '',
 '',
 "*** START OF THIS PROJECT GUTENBERG EBOOK SHAKESPEARE'S SONNETS ***"]

In [10]:
# sonnets exist between these indices 
# titles and descriptions exist outside of these indices

# use index slicing to isolate the sonnet lines - save the result to `sonnets`

# YOUR CODE HERE
sonnets = split_data[42:2620] #anything before or after these indexes are not sonnets, they're filler pages.

In [11]:
# notice how all non-sonnet lines have far fewer characters than the actual sonnet lines?
# well, let's use that observation to filter out all the non-sonnet lines
sonnets[200:240]

['  That thou among the wastes of time must go,',
 '  Since sweets and beauties do themselves forsake',
 '  And die as fast as they see others grow;',
 "    And nothing 'gainst Time's scythe can make defence",
 '    Save breed, to brave him when he takes thee hence.',
 '',
 '  XIII',
 '',
 '  O! that you were your self; but, love you are',
 '  No longer yours, than you your self here live:',
 '  Against this coming end you should prepare,',
 '  And your sweet semblance to some other give:',
 '  So should that beauty which you hold in lease',
 '  Find no determination; then you were',
 "  Yourself again, after yourself's decease,",
 '  When your sweet issue your sweet form should bear.',
 '  Who lets so fair a house fall to decay,',
 '  Which husbandry in honour might uphold,',
 "  Against the stormy gusts of winter's day",
 "  And barren rage of death's eternal cold?",
 '    O! none but unthrifts. Dear my love, you know,',
 '    You had a father: let your son say so.',
 '',
 '  XIV',
 

In [12]:
# any string with less than n_chars characters will be filtered out - save results to `filtered_sonnets`

# YOUR CODE HERE
filtered_sonnets = [text for text in sonnets if len(text) > 10 ] 


filtered_sonnets

['  From fairest creatures we desire increase,',
 "  That thereby beauty's rose might never die,",
 '  But as the riper should by time decease,',
 '  His tender heir might bear his memory:',
 '  But thou, contracted to thine own bright eyes,',
 "  Feed'st thy light's flame with self-substantial fuel,",
 '  Making a famine where abundance lies,',
 '  Thy self thy foe, to thy sweet self too cruel:',
 "  Thou that art now the world's fresh ornament,",
 '  And only herald to the gaudy spring,',
 '  Within thine own bud buriest thy content,',
 "  And tender churl mak'st waste in niggarding:",
 '    Pity the world, or else this glutton be,',
 "    To eat the world's due, by the grave and thee.",
 '  When forty winters shall besiege thy brow,',
 "  And dig deep trenches in thy beauty's field,",
 "  Thy youth's proud livery so gazed on now,",
 "  Will be a tatter'd weed of small worth held:",
 '  Then being asked, where all thy beauty lies,',
 '  Where all the treasure of thy lusty days;',
 ' 

In [13]:
# instantiate the data_cleaning_toolkit class - save result to `dctk`

# YOUR CODE HERE
dctk = data_cleaning_toolkit()

In [14]:
dir(dctk)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'char_int',
 'clean_data',
 'create_X_and_Y',
 'create_char_sequences',
 'int_char',
 'maxlen',
 'n_features',
 'next_char',
 'sequences',
 'unique_chars']

In [15]:
# use data_cleaning_toolkit to remove punctuation and to case normalize - save results to `clean_sonnets`

# YOUR CODE HERE
clean_sonnets = [ ]

for line in filtered_sonnets:
  cleaned = dctk.clean_data(line)
  clean_sonnets.append(cleaned)

In [16]:
# much better!
clean_sonnets

['from fairest creatures we desire increase',
 'that thereby beautys rose might never die',
 'but as the riper should by time decease',
 'his tender heir might bear his memory',
 'but thou contracted to thine own bright eyes',
 'feedst thy lights flame with selfsubstantial fuel',
 'making a famine where abundance lies',
 'thy self thy foe to thy sweet self too cruel',
 'thou that art now the worlds fresh ornament',
 'and only herald to the gaudy spring',
 'within thine own bud buriest thy content',
 'and tender churl makst waste in niggarding',
 'pity the world or else this glutton be',
 'to eat the worlds due by the grave and thee',
 'when forty winters shall besiege thy brow',
 'and dig deep trenches in thy beautys field',
 'thy youths proud livery so gazed on now',
 'will be a tatterd weed of small worth held',
 'then being asked where all thy beauty lies',
 'where all the treasure of thy lusty days',
 'to say within thine own deep sunken eyes',
 'were an alleating shame and thriftl

### Creating Character Sequences

In [17]:
def calc_stats(corpus):
    """
    Calculates statistics on the length of every line in the sonnets
    """
    
    # write a list comprehension that calculates each sonnets line length - save the results to `doc_lens` 

    # use NumPy to calculate and return the mean, median, std, max, min of the doc lens - all in one line of code

    # YOUR CODE HERE
    doc_len = [] #create empty list, I will iterate through each line to return the length
    for line in corpus:
      doc_len.append(len(line))
    
    print(f"Mean: ", np.mean(doc_len))

    print(f"Median: ", np.median(doc_len))

    print(f"Standard deviation: ", np.std(doc_len))

    print(f"maximum length: ", np.max(doc_len))

    print(f"minimum length: ", np.min(doc_len))

    return doc_len




In [18]:
# sonnet line length statistics 
doc_len_stats = calc_stats(clean_sonnets)

Mean:  40.882075471698116
Median:  41.0
Standard deviation:  4.050187444295222
maximum length:  57
minimum length:  27


In [19]:
# use .create_char_sequences() to create sequences


maxlen = 57 #since these documents are not extremely long, it seems reasonable to set maxlen as the max value in the dataset. 

dctk.create_char_sequences(clean_sonnets)

Created 17754 sequences.


Take a look at the `data_cleaning_toolkit_class.py` file. 

In the first four lines of code in the `create_char_sequences` method, class attributes `n_features` and `unique_chars` are created. Let's call them in the cells below.

In [20]:
# number of input features for our LSTM model
dctk.n_features

27

In [21]:
# unique characters that appear in our sonnets 
dctk.unique_chars

['u',
 ' ',
 'f',
 's',
 'y',
 'r',
 'o',
 'q',
 'i',
 'w',
 'p',
 'h',
 'z',
 'l',
 'j',
 'v',
 't',
 'x',
 'k',
 'b',
 'd',
 'e',
 'a',
 'm',
 'g',
 'n',
 'c']

----

In [22]:

X, y = dctk.create_X_and_Y()

In [23]:
# notice that our input matrix isn't a matrix - it's a rank three tensor
X.shape

(17754, 20, 27)

In $X$.shape, we see three numbers (*n1*, *n2*, *n3*). What do these numbers mean?

Well, *n1* tells us the number of samples that we have. But what about the other two?

In [24]:
# first index returns a signle sample, which we can see is a sequence 
first_sample_index = 0 
X[first_sample_index]

array([[False, False,  True, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False,  True, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False,  True, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False,  True, False, False, False],
       [False,  True, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
  

Notice that each sequence (i.e., $X[i]$ where $i$ is some index value) is `maxlen` long and has `dctk.n_features` number of features. Let's try to understand this shape better.

In [25]:
# each sequence is maxlen long and has dctk.n_features number of features
X[first_sample_index].shape

(20, 27)

**Each row corresponds to a character vector,** and there is `maxlen` number of character vectors. 

**Each column corresponds to a unique character,** and there are `dctk.n_features` number of features. 


In [26]:
# let's index for a single character vector 
first_char_vect_index = 0
X[first_sample_index][first_char_vect_index]

array([False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In [27]:
# take a look at the index to character dictionary
# if a TRUE appears in the 0th index of a character vector,
# then we know that whatever char you see below next to the 0th key 
# is the character that character vector is endcoding for
dctk.int_char

{0: 'u',
 1: ' ',
 2: 'f',
 3: 's',
 4: 'y',
 5: 'r',
 6: 'o',
 7: 'q',
 8: 'i',
 9: 'w',
 10: 'p',
 11: 'h',
 12: 'z',
 13: 'l',
 14: 'j',
 15: 'v',
 16: 't',
 17: 'x',
 18: 'k',
 19: 'b',
 20: 'd',
 21: 'e',
 22: 'a',
 23: 'm',
 24: 'g',
 25: 'n',
 26: 'c'}

In [28]:
# let's look at an example to tie it all together

seq_len_counter = 0

# index for a single sample 
for seq_of_char_vects in X[first_sample_index]:
    
    # get index with max value, which will be the one TRUE value 
    index_with_TRUE_val = np.argmax(seq_of_char_vects)
    
    print (dctk.int_char[index_with_TRUE_val])
    
    seq_len_counter+=1
    
print ("Sequence length: {}".format(seq_len_counter))

f
r
o
m
 
f
a
i
r
e
s
t
 
c
r
e
a
t
u
r
Sequence length: 20


----


### Text Generation Model




In [35]:
def sample(preds, temperature=1.0):
    """
    Helper function to sample an index from a probability array
    """
    # convert preds to array 
    preds = np.asarray(preds).astype('float64')
    # scale values 
    preds = np.log(preds) / temperature
    # exponentiate values
    exp_preds = np.exp(preds)
    # this equation should look familar to you (hint: it's an activation function)
    preds = exp_preds / np.sum(exp_preds)
    # Draw samples from a multinomial distribution
    probas = np.random.multinomial(1, preds, 1)
    # return the index that corresponds to the max probability 
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    """"
    Function invoked at the end of each epoch. Prints the text generated by our model.
    """
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    

    # randomly pick a starting index 
    # will be used to take a random sequence of chars from `text`
    start_index = random.randint(0, len(text) - dctk.maxlen - 1)
    
    # this is our seed string (i.e. input seqeunece into the model)
    generated = ''

    # start the sentence at index `start_index` and include the next` dctk.maxlen` number of chars
    sentence = text[start_index: start_index + dctk.maxlen]

    # add to generated
    generated += sentence

    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    # use model to predict what the next 40 chars should be that follow the seed string
    for i in range(40):

        # shape of a single sample in a rank 3 tensor 
        x_dims = (1, dctk.maxlen, dctk.n_features)
        # create an array of zeros with shape x_dims
        # recall that python considers zeros and boolean FALSE as the same
        x_pred = np.zeros(x_dims)

        # create a seq vector for our randomly select sequence 
        # i.e. create a numerical encoding for each char in the sequence 
        for t, char in enumerate(sentence):
            # for sample 0 in seq index t and character `char` encode a 1 (which is the same as a TRUE)
            x_pred[0, t, dctk.char_int[char]] = 1

        # next, take the seq vector and pass into model to get a prediction of what the next char should be 
        preds = model.predict(x_pred, verbose=0)[0]
        # use the sample helper function to get index for next char 
        next_index = sample(preds)
        # use look up dict to get next char 
        next_char = dctk.int_char[next_index]

        # append next char to sequence 
        sentence = sentence[1:] + next_char 
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [30]:
# need this for on_epoch_end()
text = " ".join(clean_sonnets)

In [31]:
# create callback object that will print out text generation at the end of each epoch 
# use for real-time monitoring of model performance
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

----
### Train Model



In [36]:
# build text generation model layer by layer 
# fit model

# YOUR CODE HERE
model = Sequential()

model.add(LSTM(256, input_shape=(20, 27),  #input layer
               return_sequences=True)) 

# hidden layer 1

model.add(LSTM(128, return_sequences=True))
#hidden layer 2
model.add(LSTM(128))


model.add(Dense(27, 
                activation='softmax')) #output layer

model.compile(loss='categorical_crossentropy', 
              optimizer='adam')

# fit the model

model.fit(X, y,
          batch_size=256,
          epochs=75,
          callbacks=[print_callback])


Epoch 1/75
70/70 [==============================] - 16s 193ms/step - loss: 2.8815

----- Generating text after Epoch: 0
----- Generating with seed: " praise devise who i"
 praise devise who its enrbipepninhok etyutmoeetbatt o retls
Epoch 2/75
70/70 [==============================] - 14s 199ms/step - loss: 2.8320

----- Generating text after Epoch: 1
----- Generating with seed: "rth the greater bein"
rth the greater beinh l ayna ans dotolh  bliauenerhsttduuatd
Epoch 3/75
70/70 [==============================] - 16s 233ms/step - loss: 2.8160

----- Generating text after Epoch: 2
----- Generating with seed: " i see their antique"
 i see their antiquea tttwiyoe uouh rsu r o eshpsid eut  hr 
Epoch 4/75
70/70 [==============================] - 16s 228ms/step - loss: 2.6440

----- Generating text after Epoch: 3
----- Generating with seed: "am sufficd and by a "
am sufficd and by a beiy ussit hsahtl wos mbewtevn i oie wae
Epoch 5/75
70/70 [==============================] - 16s 227ms/step - los

In [38]:
# this is the joined clean sonnet data
text

'from fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self thy foe to thy sweet self too cruel thou that art now the worlds fresh ornament and only herald to the gaudy spring within thine own bud buriest thy content and tender churl makst waste in niggarding pity the world or else this glutton be to eat the worlds due by the grave and thee when forty winters shall besiege thy brow and dig deep trenches in thy beautys field thy youths proud livery so gazed on now will be a tatterd weed of small worth held then being asked where all thy beauty lies where all the treasure of thy lusty days to say within thine own deep sunken eyes were an alleating shame and thriftless praise how much more praise deservd thy beautys use if thou couldst answer this f

In [42]:
# randomly pick a starting index 
# will be used to take a random sequence of chars from `text`
# run this cell a few times and you'll see `start_index` is random
start_index = random.randint(0, len(text) - dctk.maxlen - 1)
start_index

41905

In [43]:
# next use the randomly selected starting index to sample a sequence from the `text`

# this is our seed string (i.e., input sequence into the model)
generated = ''

# start the sentence at index `start_index` and include the next` dctk.maxlen` number of chars
sentence = text[start_index: start_index + dctk.maxlen]

# add to generated
generated += sentence

generated

' lived in me that yo'

In [44]:
# this block of code lets us know what the seed string is 
# i.e., the input sequence into the model
print('----- Generating with seed: "' + sentence + '"')
sys.stdout.write(generated)

----- Generating with seed: " lived in me that yo"
 lived in me that yo

20

In [45]:
# use model to predict what the next 40 chars should be that follow the seed string
for i in range(40):

    # shape of a single sample in a rank 3 tensor 
    x_dims = (1, dctk.maxlen, dctk.n_features)
    # create an array of zeros with shape x_dims
    # recall that python considers zeros and boolean FALSE as the same
    x_pred = np.zeros(x_dims)

    # create a seq vector for our randomly select sequence 
   
    for t, char in enumerate(sentence):
        # for sample 0 in seq index t and character `char` encode a 1 (which is the same as a TRUE)
        x_pred[0, t, dctk.char_int[char]] = 1

    # next, take the seq vector and pass into model to get a prediction of what the next char should be 
    preds = model.predict(x_pred, verbose=0)[0]
    # use the sample helper function to get index for next char 
    next_index = sample(preds)
    # use look up dict to get next char 
    next_char = dctk.int_char[next_index]

    # append next char to sequence 
    sentence = sentence[1:] + next_char 

In [46]:
# this is the seed string
generated

' lived in me that yo'

In [47]:
# these are the 40 chars that the model thinks should come after the seed stirng
sentence

'at do thou sthat not'

In [48]:
# how put it all together
generated + sentence

' lived in me that yoat do thou sthat not'

# Observations

The loss becomes acceptable at about 28 epochs, and progressively reduces per each consecutive epoch. With the lowest loss reaching 0.018, the model does a relatively good job of generating words. Adding dropout or more than 3 hidden layers did not increase performance. 